Feature Engineering (FE)
===
Better Feature better score!

install featexp by pip.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


import seaborn as sns
%matplotlib inline

In [ ]:
import random,gc
import lightgbm as lgb
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold,train_test_split,cross_validate
import warnings,random
warnings.filterwarnings('ignore')

In [ ]:
train_df = pd.read_csv('input/train.csv')
test_df = pd.read_csv('input/test.csv')

In [ ]:
features=train_df.columns[2:]

In [ ]:
import gc 
gc.collect()

# Feature Exploration



In [ ]:
from featexp import get_univariate_plots,get_trend_stats,univariate_plotter

In [ ]:
#feat=WWW_train.columns[1:]
feat=train_df.columns[1:]

In [ ]:
order=WWW_train.nunique()

In [ ]:
# explore target - var_86 relation, 
# left piture shows the trend, right shows the numbers of samples
get_univariate_plots(data=train_df[feat], target_col= 'target',features_list=['var_68'], bins= 10)
                     

In [ ]:
# Observe all the traget with all vars
i=5

feat_ob=feat[2+10*i:2+10*(i+1)]
get_univariate_plots(data=WWW_train[feat], target_col= 'target',features_list=feat_ob, bins= 10)
                     

In [ ]:
# check whether train-test is similar
train, test = train_test_split(train_df, test_size=0.33,random_state=random.randint(1,1e5))

In [ ]:
train, test = train_test_split(train_df, test_size=0.33,random_state=random.randint(1,1e5))
get_univariate_plots(data=train, target_col= 'target', data_test=test, 
                     features_list=[ 'var_68'])

In [ ]:
grouped_train, grouped_test = univariate_plotter(data=train, target_col='target', feature='var_68',
                                                 data_test=test)

In [ ]:
grouped_train, grouped_test = univariate_plotter(data=train, target_col='target', feature='var_68',
                                                 data_test=test)

In [ ]:
grouped_train

In [ ]:
stats = get_trend_stats(data=train, target_col= 'target', data_test=test)

In [ ]:
# statistics dataframe
stats

In [ ]:
# features with correlation more than 0.95
feature_1=stats[stats.Trend_correlation>0.95].Feature.values

In [ ]:
feature_1

In [ ]:
target_feature=['target']
var_features=train_df.columns.tolist()[2:]

In [ ]:
# split data into train and test sets  
train, test = train_test_split(train_df, test_size=0.33,random_state=random.randint(1,1e5))
train.head()

In [ ]:

gc.collect()

In [ ]:
# Hyperparameter grid
param_base = {
    'metric':'auc',
    'max_depth':3,
    'objective': 'binary', 
    'verbosity': 1
}

In [ ]:
param = {
    'bagging_freq': 5,          
    'bagging_fraction': 0.23,
    'bootstrap' :  True,
    'bagging_with_replacement' : True,
    'boost_from_average':'false',   
    'boost': 'gbdt',#'dart',
    'feature_fraction': 0.04,   
    'learning_rate': 0.006,     
    'max_depth': 6,
    'num_leaves': 128-50,
    'metric':'auc',
    'lambda_l1':0.01, 
    'lambda_l2':0.01,
    'min_data_in_leaf': 80,     
    'min_sum_hessian_in_leaf': 10.0,'num_leaves': 13,           
    'num_threads': 8,
    'tree_learner': 'serial',
    #'max_bin': 40,
    'objective': 'binary', 
    'is_unbalance': 'true',
    'verbosity': 1
}


In [ ]:
feature_1

In [ ]:
num_round = 1000000
features=train.columns[2:]
target=train['target']
train, test = train_test_split(train_df, test_size=0.33,random_state=random.randint(1,1e5))

trn_data=lgb.Dataset(train[features],label=train['target'])
val_data=lgb.Dataset(test[features],label=test['target'])

clf=lgb.train(param, trn_data,num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, 
              early_stopping_rounds = 1500)

In [ ]:
def get_lgb_imp_df(model,features):
    imp = pd.DataFrame(sorted(zip(model.feature_importance(),features)),
                               columns=['importance','Feature'])
    imp['importance']=imp.index.values
    return (imp)
importance_df = get_lgb_imp_df(clf,features) # get xgboost importances in dataframe
stats = pd.merge(stats, importance_df, how='left', on='Feature')
stats['importance'] = stats['importance'].fillna(0)

In [ ]:
list(stats[(stats['Trend_correlation']<0.95) & (stats['importance']<40)]['Feature'])

In [ ]:
noisy = list(stats[(stats['Trend_correlation']<0.95) & (stats['importance']<60)]['Feature'])
len(noisy)

In [ ]:
trn_data_1=lgb.Dataset(train[features].drop(noisy, axis=1),label=train['target'])
val_data_1=lgb.Dataset(test[features].drop(noisy, axis=1),label=test['target'])

clf_1=lgb.train(param, trn_data_1,num_round, valid_sets = [trn_data_1, val_data_1], verbose_eval=1000, 
              early_stopping_rounds = 1500)

In [ ]:
lgb.plot_importance(clf, max_num_features=40, importance_type='split',figsize=(12,10))

In [ ]:
plt.hist(clf.predict(test_df[features]))

In [ ]:
sub=pd.read_csv("input/sample_submission.csv")

In [ ]:
sub['target']=clf.predict(test_df[features].drop(noisy, axis=1)
sub.to_csv("output/featexp.csv",index=False)